In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
## For running trial
import sys
sys.path.insert(1, '..\\network')
from trial import BGTrial
import nengo
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
run_num = 1 ## how many times to run the network
sals = [0.1,0.2,0.3] ## list of salience values being used
n_ensembles = 512 ## number of ensembles
sp_dims = 512 ## number of dimensions for SSP encoding

rands=[11] ## random value for seed 

In [3]:
bg = BGTrial() ## initialise trial 
## for each seed, run the network 
for seed in rands:
    results = bg.run(saliences=[0.1,0.2,0.3],
                    run_num=run_num,
                    sp_dims=sp_dims,
                    version='new',
                    n_ensembles=n_ensembles,
                    n_neurons=int(200*(sp_dims/n_ensembles)),
                    relus_list=[],
                    neuron_type=nengo.neurons.LIF(), ## use spiking neurons with nengo.neurons.LIF(), or non-spiking with nengo.neurons.LIFRate()
                    plot=False, 
                    get_spikes=True,
                    verbose=False,
                    seed=seed,
                    data_format='npz',
                    data_dir='..\\data\\debug',) ## change to match desired location for saving data

Build finished in 0:03:16.                                                      
Run time: 3 seconds
Simulation finished in 0:03:43.                                                 


In [4]:
## Calculate and print out performance metrics during the timestep 10 steps before the end of each 1 second bout 
## i.e. in timestep 990, 1990 and 2990
for i in range(3):
    ## get indexes for first and last time step each second
    if i == 0:
        start = 0
    else:
        start = 1000*(i)-1
    end =  (1000*(i+1))-1
    
    ## index of minimum salience
    min_idx = np.argmin(results['input_actions'][(1000*(i+1))-10])
    ## index of maximum salience
    max_idx = np.argmax(results['input_actions'][(1000*(i+1))-10])
    ## difference between mean min and max salience coming out
    diff_out = np.asarray(results['decoded_saliences'])[start:end, max_idx].mean() - np.asarray(results['decoded_saliences'])[start:end, min_idx].mean()
    ## difference between min and max salience going in
    diff_in = results['input_actions'][(1000*(i+1))-10][max_idx] - results['input_actions'][(1000*(i+1))-10][min_idx] 
    ## has the difference between saliences been exaggerated
    Delta_diff = diff_out - diff_in

    print(Delta_diff)

0.06921783108789131
0.08028059381817892
0.09762518478317556
